In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import re
import unicodedata
from sklearn.model_selection import train_test_split
from tensorflow.python.layers.core import Dense
from tqdm import tqdm

In [2]:
path_to_zip = tf.keras.utils.get_file('spa-eng.zip', 
                                      origin='http://download.tensorflow.org/data/spa-eng.zip', 
    extract=True)
path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

In [3]:
# convert unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [4]:
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    #w = '<start> ' + w + ' <end>'
    return w

In [5]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
    lines = open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    return word_pairs

In [6]:
val =create_dataset(path_to_file, 5)
val

[['go .', 've .'],
 ['go .', 'vete .'],
 ['go .', 'vaya .'],
 ['go .', 'vayase .'],
 ['hi .', 'hola .']]

In [7]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang, target=False):
        self.lang = lang
        self.target = target
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        self.create_index()
    def create_index(self):
        for phrase in self.lang:
          self.vocab.update(phrase.split(' '))
    
        self.vocab = sorted(self.vocab)
        
        if self.target:
            self.word2idx['<pad>'] = 0
            self.word2idx['<START>'] = 1
        else:
            self.word2idx['<pad>'] = 0
    
        
        for index, word in enumerate(self.vocab):
          self.word2idx[word] = index + 1
    
        for word, index in self.word2idx.items():
          self.idx2word[index] = word

In [8]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [9]:
def load_dataset(path, num_examples):
    # creating cleaned input, output pairs
    pairs = create_dataset(path, num_examples)
    
    # index language using the class defined above    
    inp_lang = LanguageIndex(sp for en, sp in pairs)
    targ_lang = LanguageIndex((en for en, sp in pairs),target=True)
    
    # Vectorize the input and target languages
    
    # Spanish sentences
    input_tensor = [[inp_lang.word2idx[s] for s in sp.split(' ')] for en, sp in pairs]
    #print(input_tensor)
    # English sentences
    target_tensor = [[1]+[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
    target_tensor_dec = [[1]+[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar, max_length_tar_dec = max_length(input_tensor), max_length(target_tensor),\
    max_length(target_tensor_dec)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor,
                                                                maxlen=max_length_inp,
                                                                padding='post')
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor,
                                                                 maxlen=max_length_tar,
                                                                 padding='post')
    target_tensor_dec = tf.keras.preprocessing.sequence.pad_sequences(target_tensor_dec,
                                                                     maxlen=max_length_tar+1,
                                                                     padding='post')
    return input_tensor, target_tensor,target_tensor_dec, inp_lang, targ_lang, max_length_inp, max_length_tar,\
max_length_tar_dec

In [10]:
# Try experimenting with the size of that dataset
num_examples = 10000
input_tensor, target_tensor, target_tensor_dec, inp_lang, targ_lang, max_length_inp, max_length_targ, max_length_tar_dec = \
load_dataset(path_to_file, num_examples)

In [11]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = \
train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(2400, 2400, 600, 600)

In [12]:
def dataset(data1, data2, data3, buffer_size, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((data1,data2, data3)).shuffle(buffer_size)
    dataset = dataset.map(lambda x, y, z: (x,y,z, tf.size(x),tf.size(y), tf.size(z)))
    dataset = dataset.batch(batch_size)
    initializer = dataset.make_initializable_iterator()
    input_data, input_data_dec, target_data_dec, inp_len,inp_dec_len,tar_dec_len = \
    initializer.get_next()
    return input_data, input_data_dec, target_data_dec, inp_len, inp_dec_len, tar_dec_len, initializer

In [13]:
class Config:
    
    
    learning_rate = 0.001
    batch_size = 128
    buffer_size = len(input_tensor_train)//batch_size
    unit = 1024
    embedding_dim = 300
    n_epoch = 200
        
    vocab_inp_size = len(inp_lang.word2idx)
    vocab_tar_size = len(targ_lang.word2idx)
    reverse_dict_input = inp_lang.idx2word
    reverse_dict_target = targ_lang.idx2word
    max_dec_inp_len = max_length_targ
        
    cell = tf.nn.rnn_cell.BasicLSTMCell
    optimizer = tf.train.RMSPropOptimizer
    #dataset = dataset(input_tensor,target_tensor,target_tensor_dec, buffer_size, batch_size)
    #initializer = dataset.make_initializable_iterator()
    #input_data, input_data_dec, target_data_dec, inp_len,inp_dec_len,tar_dec_len = \
    #initializer.get_next()
        
    #checkpoint Path
    ckpt_dir = 'F:\AnacondaProjects\RNN-LSTM\ckpt_dir_seq'

In [14]:
def idx2token(idx, reverse_dict):
    return reverse_dict[idx]

def idx2sent(indices, reverse_dict):
    return ' '.join([idx2token(idx,reverse_dict) for idx in indices])

In [22]:
class seq2seq(object):
    
    def __init__(self, sess, config, mode):
        self.mode = mode
        self.learning_rate = config.learning_rate
        self.batch_size = config.batch_size
        self.n_epoch = config.n_epoch
        
        self.unit = config.unit
        self.embedding_dim = config.embedding_dim
        self.encoder_vocab_size = config.vocab_inp_size
        self.decoder_vocab_size = config.vocab_tar_size
        self.buffer_size = config.buffer_size
        self.cell = config.cell
        self.optimizer = config.optimizer
        
        self.ckpt_path = config.ckpt_dir
        #encoder input 
        #self.input_data = config.input_data
        #self.inp_len = config.inp_len
        # decoder input
        #self.input_data_dec = config.input_data_dec
        #self.inp_dec_len = config.inp_dec_len
        #self.tar_dec_len = config.tar_dec_len
        self.max_length_targ = config.max_dec_inp_len
        self.dec_reverse_dict = config.reverse_dict_target
        self.input_reverse_dict = config.reverse_dict_input
        # dataset initializer
        #self.initializer = config.initializer
        #sess.run(tf.global_variables_initializer())
        self.input_data, self.input_data_dec, _,self.inp_len, self.inp_dec_len, self.tar_dec_len, self.initializer  = \
        dataset(input_tensor,target_tensor,target_tensor_dec, self.buffer_size, self.batch_size)
        
    
    def encoder(self):
        
        with tf.variable_scope('encoder') as encoder:
            self.encWemb = tf.get_variable(
                'embedding',
                initializer = tf.random_uniform([self.encoder_vocab_size, self.embedding_dim]),
                dtype = tf.float32
            )
        
        self.enc_cell = self.cell(self.unit)
        
        enc_emb_input = tf.nn.embedding_lookup(self.encWemb, self.input_data, name='encoder_input')
        
        self.enc_outputs, self.enc_last_state = tf.nn.dynamic_rnn(
            cell= self.enc_cell,
            inputs = enc_emb_input,
            sequence_length = self.inp_len,
            time_major = False,
            dtype = tf.float32
        )
    
    def decoder(self):
        
        with tf.variable_scope('decoder') as decoder:
            self.decWemb = tf.get_variable(
                'embedding',
                initializer = tf.random_uniform([self.decoder_vocab_size, self.embedding_dim]),
                dtype = tf.float32
            )
        
        self.dec_cell = self.cell(self.unit)
        
        dec_emb_input = tf.nn.embedding_lookup(self.decWemb, self.input_data_dec)
        
        output_layer = Dense(self.decoder_vocab_size, name='output_projection')
        
        if self.mode == 'training':
            
            train_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_emb_input,
                                                             sequence_length=self.inp_dec_len)
            
            train_decoder = tf.contrib.seq2seq.BasicDecoder(cell=self.dec_cell,
                                                            helper=train_helper,
                                                            initial_state=self.enc_last_state,
                                                            output_layer=output_layer)
            self.train_dec_output, train_dec_last_state, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder=train_decoder,
                impute_finished=True,
                maximum_iterations=self.max_length_targ)
            
            logits = tf.identity(self.train_dec_output.rnn_output, name='logits')
            
            masks = tf.sequence_mask(self.inp_dec_len, self.max_length_targ, dtype=tf.float32,  name='masks')
            
            self.batch_loss = tf.contrib.seq2seq.sequence_loss(
                logits=logits,
                targets=self.input_data_dec,
                weights=masks,name='batch_loss'
            )
            
            self.valid_predictions = tf.identity(self.train_dec_output.sample_id, name='valid_preds')
        
        elif self.mode == 'inference':
            
            batch_size = tf.shape(self.input_data)[0:1]
            start_token = tf.ones(batch_size, dtype=tf.int32)
            
            infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                embedding=self.decWemb,
                start_tokens=start_token,
                end_token=0)
            
            infer_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = self.dec_cell,
                helper=infer_helper, 
                initial_state=self.enc_last_state,
                output_layer=output_layer)
            
            self.infer_dec_outputs, infer_dec_last_state, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder=infer_decoder,
                impute_finished=True,
                maximum_iterations=tf.reduce_max(self.inp_dec_len))
            
            self.predictions = tf.identity(self.infer_dec_outputs.sample_id, name='predictions')

    def add_training_op(self):
        self.training_op = self.optimizer(self.learning_rate).minimize(self.batch_loss)
    
    
    def saver(self, sess, var_list=None, save_path=None):
        print('Saving model at {0}'.format(save_path))
        if hasattr(self, 'training_variables'):
            var_list = self.training_variables
        saver = tf.train.Saver(var_list=var_list)
        saver.save(sess, save_path, write_meta_graph=False)
        
    def restore(self, sess, var_list=None,save_path=None):
        if hasattr(self, 'training_variables'):
            var_list = self.training_variables
        self.restorer = tf.train.Saver(var_list)
        self.restorer.restore(sess, save_path)
        print('Restore Finished!')
        
    def summary(self):
        summary_writer = tf.summary.FileWriter(
            logdir = self.ckpt_path,
            graph = tf.get_default_graph()
        )
        
    def build(self):
        self.encoder()
        self.decoder()
        
    
    def train(self,from_scratch=False, load_ckpt = None, save_path=None):
        
        if from_scratch is False and os.path.isfile(load_ckpt):
            self.restore(sess, save_path=load_ckpt)
        
        
        self.build()
        self.add_training_op()
        sess.run(tf.global_variables_initializer())
        
        loss_history = []
        for epoch in tqdm(range(self.n_epoch)):
            all_preds = []
            all_output = []
            per_epoch_inp = []
            per_epoch_inp_dec = []
            epoch_loss = 0
            sess.run([self.initializer.initializer])
            try:
                while True:
                    output,batch_pred, batch_loss, _, inp_data, inp_data_dec = sess.run([self.train_dec_output.sample_id,self.valid_predictions, self.batch_loss, self.training_op,\
                                                          self.input_data,self.input_data_dec])
                    epoch_loss += batch_loss
                    all_preds.append(batch_pred)
                    all_output.append(output)
                    per_epoch_inp.append(inp_data)
                    per_epoch_inp_dec.append(inp_data_dec)
            except tf.errors.OutOfRangeError:
                pass
        
            loss_history.append(epoch_loss)
            if epoch%100 == 0:
                
                print('Epoch', epoch)
                for input_batch, target_batch, batch_pred in zip(per_epoch_inp,per_epoch_inp_dec,all_output):
                    for input_sent, target_sent, pred in zip(input_batch,target_batch, batch_pred):
                        print('\tinput sent:', idx2sent(input_sent,self.input_reverse_dict))
                        print('\tprediction:', idx2sent(pred,self.dec_reverse_dict))
                        print('\tTarget:', idx2sent(target_sent,self.dec_reverse_dict))
            
            #print('\t outputs: ',all_output)
            #print('\tBatch Predictions',batch_pred)
            print('\tepoch loss: {0}\n'.format(epoch_loss))

        if save_path:
            self.saver(sess,save_path=self.ckpt_path)
        return loss_history
    
    def inference(self,load_ckpt):
        self.build()
        self.restore(sess, save_path=load_ckpt)
        
        batch_preds = []
        batch_inputs = []
        batch_inputs_dec= []
        sess.run([self.initializer.initializer])
        try:
            while True:
                output, inp_data, inp_data_dec = sess.run([self.infer_dec_outputs.sample_id,\
                                                           self.input_data,self.input_data_dec])
                batch_preds.append(output)
                batch_inputs.append(inp_data)
                batch_inputs_dec.append(inp_data_dec)
        except tf.errors.OutOfRangeError:
            pass
                
        for input_batch, target_batch, batch_pred in zip(batch_inputs,batch_inputs_dec, batch_preds):
            for input_sent, target_sent, pred in zip(input_batch,target_batch, batch_pred):
                print('\tinput sent', idx2sent(input_sent,self.input_reverse_dict))
                print('\tprediction', idx2sent(pred,self.dec_reverse_dict))
                print('\tTarget:', idx2sent(target_sent,self.dec_reverse_dict))
                        
        
        


In [16]:
tf.reset_default_graph()
with tf.Session() as sess:
    config = Config()
    model = seq2seq(sess, config, mode='training')
    model.train(from_scratch=True, save_path=model.ckpt_path)
    print('Training model built!')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

Epoch 0
	input sent: parad ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked liked let let let let let
	Target: <START> stop <START> <pad> <pad> <pad> <pad>
	input sent: vayase . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked liked scream god god god god
	Target: <START> go . <pad> <pad> <pad> <pad>
	input sent: para ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked liked let let let let let
	Target: <START> stop <START> <pad> <pad> <pad> <pad>
	input sent: hola . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked scream scream god god god god
	Target: <START> hi . <pad> <pad> <pad> <pad>
	input sent: incendio ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked god let let let god god
	Target: <START> fire <START> <pad> <pad> <pad> <pad>
	input sent: espera ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked scream scream let let let let
	Target: <START> wait <START> <pad> <pad> <pad> <pad>
	input sent: auxilio ! <

	input sent: de ninguna manera ! <pad> <pad> <pad> <pad> <pad>
	prediction: liked god god liked let let god
	Target: <START> no way <START> <pad> <pad> <pad>
	input sent: el corrio . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked god god god god god god
	Target: <START> he ran . <pad> <pad> <pad>
	input sent: ¿ en serio ? <pad> <pad> <pad> <pad> <pad>
	prediction: liked god let let let let god
	Target: <START> really ? <pad> <pad> <pad> <pad>
	input sent: ganamos . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked scream scream let let let god
	Target: <START> we won . <pad> <pad> <pad>
	input sent: ¿ la verdad ? <pad> <pad> <pad> <pad> <pad>
	prediction: liked god let let let let god
	Target: <START> really ? <pad> <pad> <pad> <pad>
	input sent: gracias ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked god god god god god god
	Target: <START> thanks . <pad> <pad> <pad> <pad>
	input sent: ni en pedo ! <pad> <pad> <pad> <pad> <pad>
	prediction: scream god

	prediction: liked scream let let let let god
	Target: <START> go slow . <pad> <pad> <pad>
	input sent: orale ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked liked god scream god god god
	Target: <START> go away . <pad> <pad> <pad>
	input sent: hasta luego ! <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked god god god god god god
	Target: <START> goodbye <START> <pad> <pad> <pad> <pad>
	input sent: hasta la vista . <pad> <pad> <pad> <pad> <pad>
	prediction: liked god god god god god god
	Target: <START> goodbye <START> <pad> <pad> <pad> <pad>
	input sent: largo ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked liked god scream god god god
	Target: <START> go away <START> <pad> <pad> <pad>
	input sent: sal . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked liked liked liked let let let
	Target: <START> get out . <pad> <pad> <pad>
	input sent: se agradable . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction: liked scream scream god god god god


...

	epoch loss: 4.47721990942955



 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [22:24<01:10, 70.78s/it]

	epoch loss: 3.6236866861581802



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [23:27<00:00, 70.35s/it]


Saving model at F:\AnacondaProjects\RNN-LSTM\ckpt_dir_seq
Training model built!


In [23]:
tf.reset_default_graph()
with tf.Session() as sess:
    config = Config()
    model = seq2seq(sess, config, mode='inference')
    model.inference(model.ckpt_path)
    print('inference model built!')

INFO:tensorflow:Restoring parameters from F:\AnacondaProjects\RNN-LSTM\ckpt_dir_seq
Restore Finished!
	input sent parad ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> stop <START> <pad> <pad> <pad> <pad>
	input sent vaya . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> go . <pad> <pad> <pad> <pad>
	input sent vayase . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> go . <pad> <pad> <pad> <pad>
	input sent disparad ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> fire <START> <pad> <pad> <pad> <pad>
	input sent socorro ! auxilio ! <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> help <START> <pad> <pad> <pa

	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> listen . <pad> <pad> <pad> <pad>
	input sent sali . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i left . <pad> <pad> <pad>
	input sent perdi . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i lost . <pad> <pad> <pad>
	input sent mangos ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> no way <START> <pad> <pad> <pad>
	input sent fuego ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> fire <START> <pad> <pad> <pad> <pad>
	input sent ni en pedo ! <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> no way <START> <pad> <pad> <pa

	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> go away . <pad> <pad> <pad>
	input sent andale . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> come on . <pad> <pad> <pad>
	input sent largo ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> go away . <pad> <pad> <pad>
	input sent orale ! <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> come on . <pad> <pad> <pad>
	input sent vete a casa . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> go home . <pad> <pad> <pad>
	input sent sali . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> get out . <pad> <pad> <pad>
	input sent vete d

	Target: <START> it s ok . <pad> <pad>
	input sent agarra fuertemente . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> hold on . <pad> <pad> <pad>
	input sent estoy en forma . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i m fit . <pad> <pad>
	input sent salte . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> get out . <pad> <pad> <pad>
	input sent estoy mojada . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i m wet . <pad> <pad>
	input sent abraza a tom . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> hug tom . <pad> <pad> <pad>
	input sent dormi . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> 

	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> we know . <pad> <pad> <pad>
	input sent ¿ por que no ? <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> why not ? <pad> <pad> <pad>
	input sent apartate . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> back off . <pad> <pad> <pad>
	input sent ¿ quien gano ? <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> who won ? <pad> <pad> <pad>
	input sent ¿ quien corrio ? <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> who ran ? <pad> <pad> <pad>
	input sent lavate las manos . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> wash up . <pad> <pad> <pad>
	input sent ¿ quien corria ?

...

	Target: <START> come on home . <pad> <pad>
	input sent ¿ tu roncas ? <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> do you snore ? <pad> <pad>
	input sent hazlo el lunes . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> do it monday . <pad> <pad>
	input sent ¿ bebeis ? <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> do you drink ? <pad> <pad>
	input sent venid aqui arriba . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> come up here . <pad> <pad>
	input sent regresa . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> come on back . <pad> <pad>
	input sent ¿ duele ? <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <S

	input sent el es celoso . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> he s jealous . <pad> <pad>
	input sent el no esta dentro . <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> he is not in . <pad>
	input sent esta de morros . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> he is cranky . <pad> <pad>
	input sent ayuda a tom . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> help tom out . <pad> <pad>
	input sent coge . <pad> <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> here you are . <pad> <pad>
	input sent aqui tienes un boligrafo . <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> here s a 

	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i can t talk . <pad>
	input sent estoy rezando . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i am praying . <pad> <pad>
	input sent yo comi manzanas . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i ate apples . <pad> <pad>
	input sent podria caminar . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i could walk . <pad> <pad>
	input sent te admiro . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i admire you . <pad> <pad>
	input sent me disculpe . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i apologized . <pad> <pad> <pad>
	input sent yo n

	input sent necesito pintura . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i need paint . <pad> <pad>
	input sent me gusta eso . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i like these . <pad> <pad>
	input sent yo necesito agua . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i need water . <pad> <pad>
	input sent tenia hambre . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i was hungry . <pad> <pad>
	input sent me encantan los viajes . <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i love trips . <pad> <pad>
	input sent te necesitaba . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>

	input sent estoy descalzo . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i m barefoot . <pad> <pad>
	input sent soy medica . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i m a doctor . <pad>
	input sent estoy dubitativo . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i m doubtful . <pad> <pad>
	input sent he avisado a tom . <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i warned tom . <pad> <pad>
	input sent soy camarero . <pad> <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <START> i m a waiter . <pad>
	input sent hice una apuesta . <pad> <pad> <pad> <pad> <pad>
	prediction <START> <START> <START> <START> <START> <START> <START>
	Target: <ST